In [7]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import cna
import multianndata as mad

# For Aparna's Import
import scipy.io

# For reproducibility
np.random.seed(0) 

In [5]:
def run_cna(variable, cna_obj, covs = None):
    if covs is not None:
        cna_res = cna.tl._association.association(cna_obj, #dataset 
                                                  cna_obj.samplem[variable], #phenotype
            #                                       batches=d.samplem.processing_batch, #batches
                                                  covs = cna_obj.samplem[covs],
                                                  Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                                  ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                            #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                                 )
    else:
        cna_res = cna.tl._association.association(cna_obj, #dataset 
                                                  cna_obj.samplem[variable], #phenotype
            #                                       batches=d.samplem.processing_batch, #batches
                                                 # covs = d.samplem[covs]
                                                  Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                                  ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                            #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                                 )
        
    return cna_res
def cna_test(meta_df, harmony_df, umap_df, variable, covars = None):
    print(meta_df.shape)
    print(harmony_df.shape)
    print(umap_df.shape)
    cna_object = mad.MultiAnnData(X=harmony_df, obs=meta_df, sampleid="sample")
    if covars is not None:   
        cna_object.obs_to_sample(covars + [variable])
        umap_df.index = cna_object.obs.index
        cna_object.obsm['X_umap'] = umap_df
        cna.pp.knn(cna_object)
        np.random.seed(0) 
        final_res = run_cna(variable, cna_object, covs = covars)
    else:
        cna_object.obs_to_sample([variable])
        umap_df.index = cna_object.obs.index
        cna_object.obsm['X_umap'] = umap_df
        cna.pp.knn(cna_object)
        np.random.seed(0) 
        final_res = run_cna(variable, cna_object, covs = None)
    print(final_res.k)
    print(final_res.ks)
    print('p =', final_res.p, ',', final_res.k, 'PCs used')
    print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(final_res.k, final_res.r2))
    return final_res

# AGE

## Single Cell

In [4]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_meta_age.csv')
print(meta.shape)

NameError: name 'pd' is not defined

In [4]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_harmony_age.csv')
print(harmony.shape)

(12591, 20)


In [5]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_umap_age.csv')
print(umap.shape)

(12591, 2)


In [6]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.592014,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.379611,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.150366,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.503684,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.174577,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [7]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [8]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Age, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 63.43772506713867
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 43.07426071166992
	20th percentile R2(t,t-1): 0.750810730457306
	taking step 3
	median kurtosis: 32.59674835205078
	20th percentile R2(t,t-1): 0.9043575286865234
	taking step 4
	median kurtosis: 24.89748191833496
	20th percentile R2(t,t-1): 0.9453570485115051
	taking step 5
	median kurtosis: 19.845382690429688
	20th percentile R2(t,t-1): 0.9633108735084533
	taking step 6
	median kurtosis: 16.429214477539062
	20th percentile R2(t,t-1): 0.9762293457984924
	taking step 7
	median kurtosis: 13.926634788513184
	20th percentile R2(t,t-1): 0.9843466401100158
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
13
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.23167683231676833 , 13 PCs used
total r^2 between top 13 NAM PCs and outcome is 0.14


In [10]:
sc_uni['Age'] = res.p

# SEX

## Single Cell

In [11]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_meta_sex.csv')
print(meta.shape)

(12591, 39)


In [12]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_harmony_sex.csv')
print(harmony.shape)

(12591, 20)


In [13]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_umap_sex.csv')
print(umap.shape)

(12591, 2)


In [14]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.592014,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.379611,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.150366,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.503684,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.174577,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [15]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [16]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [17]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Sex, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 63.43772506713867
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 43.07426071166992
	20th percentile R2(t,t-1): 0.750810730457306
	taking step 3
	median kurtosis: 32.59674835205078
	20th percentile R2(t,t-1): 0.9043575286865234
	taking step 4
	median kurtosis: 24.89748191833496
	20th percentile R2(t,t-1): 0.9453570485115051
	taking step 5
	median kurtosis: 19.845382690429688
	20th percentile R2(t,t-1): 0.9633108735084533
	taking step 6
	median kurtosis: 16.429214477539062
	20th percentile R2(t,t-1): 0.9762293457984924
	taking step 7
	median kurtosis: 13.926634788513184
	20th percentile R2(t,t-1): 0.9843466401100158
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.36866313368663134 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.03


In [19]:
sc_uni['Sex'] = res.p

# RESPONSE

## Single Cell

In [20]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_meta_response.csv')
print(meta.shape)

(9514, 36)


In [21]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_harmony_response.csv')
print(harmony.shape)

(9514, 20)


In [22]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_umap_response.csv')
print(umap.shape)

(9514, 2)


In [26]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]',  'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_Einstein,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.737171,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.319563,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.256838,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.641104,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.184163,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [27]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [28]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [29]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Responder_Status, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 50.739593505859375
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 35.00537109375
	20th percentile R2(t,t-1): 0.7539314031600952
	taking step 3
	median kurtosis: 26.470333099365234
	20th percentile R2(t,t-1): 0.9063006520271302
	taking step 4
	median kurtosis: 20.533781051635742
	20th percentile R2(t,t-1): 0.9482423782348632
	taking step 5
	median kurtosis: 16.708977699279785
	20th percentile R2(t,t-1): 0.9657996892929077
	taking step 6
	median kurtosis: 13.999242782592773
	20th percentile R2(t,t-1): 0.978196132183075
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.067993200679932 , 10 PCs used
total r^2 between top 10 NAM PCs and outcome is 0.17


In [30]:
sc_uni['Responder_Status'] = res.p

# RACE

## Single Cell

In [31]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_meta_race.csv')
print(meta.shape)

(12591, 39)


In [32]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_harmony_race.csv')
print(harmony.shape)

(12591, 20)


In [33]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/age_sex_response_race/sc_umap_race.csv')
print(umap.shape)

(12591, 2)


In [34]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.592014,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.379611,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.150366,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.503684,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.174577,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [35]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [36]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [43]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.981001899810019 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


In [42]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[A][B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.83991600839916 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


In [41]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.34056594340565943 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.13


In [40]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B][AI]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6743325667433256 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.01


In [39]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[B][W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
2
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8332166783321668 , 2 PCs used
total r^2 between top 2 NAM PCs and outcome is 0.01


In [44]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[U]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.19008099190080993 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.16


In [45]:
sc_uni['Race'] = res.p

In [38]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Race_[W]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6525347465253475 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.11


# CASE/CONTROL

## Single Cell

In [3]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna/b_plasma/sc_b_plasma_casecontrol_meta.csv')
meta = meta.join(pd.get_dummies(meta['Race'], prefix = "Race", prefix_sep = "", drop_first = False))
meta = meta.join(pd.get_dummies(meta['Site'], prefix = "Site", prefix_sep = "", drop_first = False))
meta = meta.join(pd.get_dummies(meta['Responder.Status'], prefix = "", prefix_sep = "", drop_first = False))
print(meta.shape)

(12644, 83)


In [4]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna/b_plasma/sc_b_plasma_casecontrol_harmony.csv')
print(harmony.shape)

(12644, 20)


In [5]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna/b_plasma/sc_b_plasma_casecontrol_umap.csv')
print(umap.shape)

(12644, 2)


In [6]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="individual")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Type_numeric'])
d.samplem.head()

['cell' 'sample' 'Annot.separate' 'dataset' 'Site' 'Type' 'broad.type'
 'doublet_classification' 'arnon_cluster' 'annotation' 'AMP.ID'
 'Clinical.Response.at.12.weeks' 'Clinical.Response.at.26.weeks'
 'Clinical.Response.at.52.weeks' 'Responder.Status' 'Race'
 'processing.date' 'XTR.WTA.batch' 'DASH.batch' 'index'
 'seq.submission.date' 'Flow.cell' 'biopsy.length.from.site.mm']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Type_numeric
individual,
134,2.0
137,2.0
138,2.0
139,2.0
140,2.0


In [7]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [8]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Type_numeric, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 73.8285903930664
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 51.8582878112793
	20th percentile R2(t,t-1): 0.749199628829956
	taking step 3
	median kurtosis: 40.53631591796875
	20th percentile R2(t,t-1): 0.9042156100273132
	taking step 4
	median kurtosis: 32.24089431762695
	20th percentile R2(t,t-1): 0.9449354410171509
	taking step 5
	median kurtosis: 26.5614013671875
	20th percentile R2(t,t-1): 0.9629530072212219
	taking step 6
	median kurtosis: 22.408252716064453
	20th percentile R2(t,t-1): 0.974814236164093
	taking step 7
	median kurtosis: 19.055797576904297
	20th percentile R2(t,t-1): 0.9831657409667969
	taking step 8
	median kurtosis: 16.621466636657715
	20th percentile R2(t,t-1): 0.9878508448600769
stopping after 8 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.00029997000299970003 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.36


In [10]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna/b_plasma/sc_b_plasma_casecontrol.kept.csv", res.kept, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna/b_plasma/sc_b_plasma_casecontrol.ncorrs.csv", res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna/b_plasma/sc_b_plasma_casecontrol.fdrs.csv", res.fdrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna/b_plasma/sc_b_plasma_casecontrol.thresh.csv", [res.fdr_5p_t, res.fdr_10p_t], delimiter=",")

# SITE

## Single Cell

In [47]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_meta_site.csv')
print(meta.shape)

(12591, 39)


In [48]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_harmony_site.csv')
print(harmony.shape)

(12591, 20)


In [49]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_umap_site.csv')
print(umap.shape)

(12591, 2)


In [50]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.592014,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.379611,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.150366,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.503684,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.174577,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [51]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [52]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [53]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Cincinnati, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 63.43772506713867
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 43.07426071166992
	20th percentile R2(t,t-1): 0.750810730457306
	taking step 3
	median kurtosis: 32.59674835205078
	20th percentile R2(t,t-1): 0.9043575286865234
	taking step 4
	median kurtosis: 24.89748191833496
	20th percentile R2(t,t-1): 0.9453570485115051
	taking step 5
	median kurtosis: 19.845382690429688
	20th percentile R2(t,t-1): 0.9633108735084533
	taking step 6
	median kurtosis: 16.429214477539062
	20th percentile R2(t,t-1): 0.9762293457984924
	taking step 7
	median kurtosis: 13.926634788513184
	20th percentile R2(t,t-1): 0.9843466401100158
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.6341365863413658 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.01


In [54]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Einstein, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.05849415058494151 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.24


In [55]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_JHU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.48595140485951405 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.12


In [56]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Michigan, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
5
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8651134886511349 , 5 PCs used
total r^2 between top 5 NAM PCs and outcome is 0.02


In [57]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_MUSC, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8932106789321068 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.00


In [58]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Northwell, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.263973602639736 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.13


In [59]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_NYU, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
13
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7042295770422957 , 13 PCs used
total r^2 between top 13 NAM PCs and outcome is 0.10


In [60]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_Rochester, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.3705629437056294 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.13


In [61]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_Site_Texas Tech'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0410958904109589 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.35


In [65]:
sc_uni['Site'] = res.p

In [62]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCLA, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
19
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.1546845315468453 , 19 PCs used
total r^2 between top 19 NAM PCs and outcome is 0.27


In [63]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSD, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
4
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7188281171882812 , 4 PCs used
total r^2 between top 4 NAM PCs and outcome is 0.03


In [64]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Site_UCSF, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
1
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.5671432856714329 , 1 PCs used
total r^2 between top 1 NAM PCs and outcome is 0.01


# FIRST BIOPSY

## Single Cell

In [66]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_meta_firstbiop.csv')
print(meta.shape)

(12591, 39)


In [67]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_harmony_firstbiop.csv')
print(harmony.shape)

(12591, 20)


In [68]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_umap_firstbiop.csv')
print(umap.shape)

(12591, 2)


In [69]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.592014,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.379611,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.150366,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.503684,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.174577,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [70]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [71]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [73]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.First_biop, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 63.43772506713867
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 43.07426071166992
	20th percentile R2(t,t-1): 0.750810730457306
	taking step 3
	median kurtosis: 32.59674835205078
	20th percentile R2(t,t-1): 0.9043575286865234
	taking step 4
	median kurtosis: 24.89748191833496
	20th percentile R2(t,t-1): 0.9453570485115051
	taking step 5
	median kurtosis: 19.845382690429688
	20th percentile R2(t,t-1): 0.9633108735084533
	taking step 6
	median kurtosis: 16.429214477539062
	20th percentile R2(t,t-1): 0.9762293457984924
	taking step 7
	median kurtosis: 13.926634788513184
	20th percentile R2(t,t-1): 0.9843466401100158
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving
computing SVD


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_association.py:57: UserWarning: data supported use of 20 NAM PCs, which is the maximum considered. Consider allowing more PCs by using the "ks" argument.
  warnings.warn(('data supported use of {} NAM PCs, which is the maximum considered. '+\


performing association test
computing neighborhood-level FDRs
20
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.061993800619938005 , 20 PCs used
total r^2 between top 20 NAM PCs and outcome is 0.21


In [76]:
sc_uni['First_biop'] = res.p

# PREDNISONE USE

## Single Cell

In [85]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_meta_preduse.csv')
print(meta.shape)

(12591, 39)


In [86]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_harmony_preduse.csv')
print(harmony.shape)

(12591, 20)


In [87]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/first_biop_pred_use_site/sc_umap_preduse.csv')
print(umap.shape)

(12591, 2)


In [88]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.592014,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.379611,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.150366,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.503684,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.174577,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [89]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [91]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [93]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Pred_use, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.0072992700729927005 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.20


In [94]:
sc_uni['Prednisone_use'] = res.p

# CHRONICITY

## Single Cell

In [8]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/chronicity/sc_meta.csv')
print(meta.shape)

(11729, 39)


In [9]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/chronicity/sc_harmony.csv')
print(harmony.shape)

(11729, 20)


In [10]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/chronicity/sc_umap.csv')
print(umap.shape)

(11729, 2)


In [11]:
res = cna_test(meta,harmony, umap, 'Final_Chronicity', covars = ['First_biop', 'Responder_Status'])

(11729, 39)
(11729, 20)
(11729, 2)
['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies
computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 57.85556411743164
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 38.710750579833984
	20th percentile R2(t,t-1): 0.7539346814155579
	taking step 3
	median kurtosis: 29.464487075805664
	20th percentile R2(t,t-1): 0.9061028361320496
	taking step 4
	median kurtosis: 22.658470153808594
	20th percentile R2(t,t-1): 0.9470478892326355
	taking step 5
	median kurtosis: 17.95610809326172
	20th percentile R2(t,t-1): 0.9645543694496155
	taking step 6
	median kurtosis: 15.00442123413086
	20th percentile R2(t,t-1): 0.976597785949707
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
13
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.39976002399760024 , 13 PCs used
total r^2 between top 13 NAM PCs and outcome is 0.16


# ACTIVITY

## Single Cell

In [95]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/activity/sc_meta.csv')
print(meta.shape)

(11729, 39)


In [96]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/activity/sc_harmony.csv')
print(harmony.shape)

(11729, 20)


In [97]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/activity/sc_umap.csv')
print(umap.shape)

(11729, 2)


In [98]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.541744,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.419240,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.104933,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.454382,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.205500,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [99]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [100]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [101]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem.Final_Activity, #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 57.85556411743164
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 38.710750579833984
	20th percentile R2(t,t-1): 0.7539346814155579
	taking step 3
	median kurtosis: 29.464487075805664
	20th percentile R2(t,t-1): 0.9061028361320496
	taking step 4
	median kurtosis: 22.658470153808594
	20th percentile R2(t,t-1): 0.9470478892326355
	taking step 5
	median kurtosis: 17.95610809326172
	20th percentile R2(t,t-1): 0.9645543694496155
	taking step 6
	median kurtosis: 15.00442123413086
	20th percentile R2(t,t-1): 0.976597785949707
stopping after 6 steps
covariate-adjusted NAM not found; computing and saving


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing SVD
performing association test
computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.35926407359264073 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.17


In [102]:
sc_uni['Activity'] = res.p

# ISN

## Single Cell

In [103]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/ISN/sc_meta.csv')
print(meta.shape)

(12591, 39)


In [104]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/ISN/sc_harmony.csv')
print(harmony.shape)

(12591, 20)


In [105]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/ISN/sc_umap.csv')
print(umap.shape)

(12591, 2)


In [106]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['Sex', 'Age', 'Final_Chronicity', 'Final_Activity', 'First_biop', 'Pred_use', 'Responder_Status', 'Race_[A]',
       'Race_[A][B]', 'Race_[B]', 'Race_[B][AI]', 'Race_[B][W]', 'Race_[U]',
       'Race_[W]', 'Final_ISN_[III]', 'Final_ISN_[III][V]', 'Final_ISN_[IV]',
       'Final_ISN_[IV][V]', 'Final_ISN_[V]', 'Final_Site_Cincinnati',
       'Final_Site_Einstein', 'Final_Site_JHU', 'Final_Site_Michigan',
       'Final_Site_MUSC', 'Final_Site_Northwell', 'Final_Site_NYU',
       'Final_Site_Rochester', 'Final_Site_Texas Tech', 'Final_Site_UCLA',
       'Final_Site_UCSD', 'Final_Site_UCSF'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,Sex,Age,Final_Chronicity,Final_Activity,First_biop,Pred_use,Responder_Status,Race_[A],Race_[A][B],Race_[B],...,Final_Site_JHU,Final_Site_Michigan,Final_Site_MUSC,Final_Site_Northwell,Final_Site_NYU,Final_Site_Rochester,Final_Site_Texas Tech,Final_Site_UCLA,Final_Site_UCSD,Final_Site_UCSF
sample,,,,,,,,,,,,,,,,,,,,,
AMPSLEkid_cells_0134,1.0,0.592014,6.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0137,1.0,-0.379611,3.0,5.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0138,1.0,0.150366,0.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0139,1.0,0.503684,3.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AMPSLEkid_cells_0140,1.0,-1.174577,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [107]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [108]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


In [109]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[III]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 63.43772506713867
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 43.07426071166992
	20th percentile R2(t,t-1): 0.750810730457306
	taking step 3
	median kurtosis: 32.59674835205078
	20th percentile R2(t,t-1): 0.9043575286865234
	taking step 4
	median kurtosis: 24.89748191833496
	20th percentile R2(t,t-1): 0.9453570485115051
	taking step 5
	median kurtosis: 19.845382690429688
	20th percentile R2(t,t-1): 0.9633108735084533
	taking step 6
	median kurtosis: 16.429214477539062
	20th percentile R2(t,t-1): 0.9762293457984924
	taking step 7
	median kurtosis: 13.926634788513184
	20th percentile R2(t,t-1): 0.9843466401100158
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
12
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.8595140485951405 , 12 PCs used
total r^2 between top 12 NAM PCs and outcome is 0.08


In [110]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[III][V]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
7
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.7496250374962504 , 7 PCs used
total r^2 between top 7 NAM PCs and outcome is 0.05


In [111]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[IV]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
5
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.15508449155084492 , 5 PCs used
total r^2 between top 5 NAM PCs and outcome is 0.08


In [114]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[IV][V]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

performing association test
computing neighborhood-level FDRs
17
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.08299170082991701 , 17 PCs used
total r^2 between top 17 NAM PCs and outcome is 0.20


In [115]:
sc_uni['ISN'] = res.p

In [ ]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['Final_ISN_[V]'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

In [117]:
pd.DataFrame.from_dict(sc_uni, orient='index').T.to_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/univariate_cna.csv')

# INJURED PT

## Single Celll

In [3]:
meta = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/injured_pt/sc_meta.csv')
print(meta.shape)

(12591, 16)


In [4]:
harmony = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/injured_pt/sc_harmony.csv')
print(harmony.shape)

(12591, 20)


In [5]:
umap = pd.read_csv('/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/injured_pt/sc_umap.csv')
print(umap.shape)

(12591, 2)


In [6]:
# Start by making anndata object
d = mad.MultiAnnData(X=harmony, obs=meta, sampleid="sample")
#d.obs_to_sample(['Type', 'Site',
#                 'Responder_Status', 'Sex',
#                 'Age', 'Race',
#                'Ethnicity', 'ISN',
#               'Activity', "Chronicity",
#                 'processing_batch'])
d.obs_to_sample(['injured_pt_prop', 'Final_Chronicity'])
d.samplem.head()

['cell' 'sample' 'final_annotation' 'Responder.Status' 'Race' 'Final_ISN'
 'Type' 'Final_Site']
consider casting to numeric types where appropriate, and
consider re-coding text-valued columns with pandas.get_dummies


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/multianndata/core.py:17: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  super().__init__(*args, **kwargs)
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


,injured_pt_prop,Final_Chronicity
sample,,
AMPSLEkid_cells_0134,16.551724,6.0
AMPSLEkid_cells_0137,14.341498,3.0
AMPSLEkid_cells_0138,5.629663,0.0
AMPSLEkid_cells_0139,4.405520,3.0
AMPSLEkid_cells_0140,6.399412,1.0


In [7]:
umap.index = d.obs.index
d.obsm['X_umap'] = umap

In [8]:
np.random.seed(0) 
cna.pp.knn(d)

computing default knn graph


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
np.random.seed(0) 
res = cna.tl._association.association(d, #dataset 
                                      d.samplem['injured_pt_prop'], #phenotype
#                                       batches=d.samplem.processing_batch, #batches
                                      Nnull=10000, # number of null permutations to do (defaults to only 1e3)
                                      ks=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # I asked the method to consider up to 10 PCs because
                                                                #it chose the max number of PCs it considered the default set of [1,2,3,4]
                                     )
print(res.k)
print(res.ks)
print('p =', res.p, ',', res.k, 'PCs used')
print('total r^2 between top {} NAM PCs and outcome is {:.2f}'.format(res.k, res.r2))

qcd NAM not found; computing and saving
	taking step 1
	median kurtosis: 63.43772506713867
	20th percentile R2(t,t-1): nan
	taking step 2
	median kurtosis: 43.07426071166992
	20th percentile R2(t,t-1): 0.750810730457306
	taking step 3
	median kurtosis: 32.59674835205078
	20th percentile R2(t,t-1): 0.9043575286865234
	taking step 4
	median kurtosis: 24.89748191833496
	20th percentile R2(t,t-1): 0.9453570485115051
	taking step 5
	median kurtosis: 19.845382690429688
	20th percentile R2(t,t-1): 0.9633108735084533
	taking step 6
	median kurtosis: 16.429214477539062
	20th percentile R2(t,t-1): 0.9762293457984924
	taking step 7
	median kurtosis: 13.926634788513184
	20th percentile R2(t,t-1): 0.9843466401100158
stopping after 7 steps
covariate-adjusted NAM not found; computing and saving
computing SVD
performing association test


/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:79: UserWarning: only one unique batch supplied to qc
  warnings.warn('only one unique batch supplied to qc')
/PHShome/ssg34/.conda/envs/plswork/lib/python3.9/site-packages/cna/tools/_nam.py:101: UserWarning: only one unique batch supplied to prep
  warnings.warn('only one unique batch supplied to prep')


computing neighborhood-level FDRs
14
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
p = 0.4340565943405659 , 14 PCs used
total r^2 between top 14 NAM PCs and outcome is 0.13


In [10]:
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/injured_pt/sc_injured_pt_ncorr.csv", 
               res.ncorrs, delimiter=",")
np.savetxt("/data/srlab/ssg34/SLE_kidney_v2/data/cna_new/b_plasma/injured_pt/sc_injured_pt_fdrs.csv", 
               res.fdrs, delimiter=",")